In [115]:
from pathlib import Path
from tira.rest_api_client import Client
from tira.third_party_integrations import get_output_directory
import pandas as pd

In [51]:
tira = Client()

    # loading train data
text_train = tira.pd.inputs(
        "nlpbuw-fsu-sose-24", "authorship-verification-train-20240408-training"
)
targets_train = tira.pd.truths(
        "nlpbuw-fsu-sose-24", "authorship-verification-train-20240408-training"
)
 # loading validation data (automatically replaced by test data when run on tira)
text_validation = tira.pd.inputs(
        "nlpbuw-fsu-sose-24", "authorship-verification-validation-20240408-training"
)
targets_validation = tira.pd.truths(
        "nlpbuw-fsu-sose-24", "authorship-verification-validation-20240408-training"
)

In [52]:
print(text_train)

        id  \
0     1253   
1      748   
2     1415   
3      333   
4      515   
...    ...   
1769  1412   
1770  1609   
1771  1079   
1772  1811   
1773  1406   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [53]:
pd.set_option('display.max_colwidth', 50)
concat_df = pd.concat([text_train, targets_train['generated']], axis=1)
print(concat_df)

        id                                               text  generated
0     1253  Kamala Harris to promote 'America is back' mes...          0
1      748  Tampa Bay Buccaneers Odds & Betting Lines\n\nT...          0
2     1415  Kamala Harris Tackles Root Causes of Migration...          1
3      333  Twitter Permanently Bans President Trump's Acc...          1
4      515  Tiger Woods Hospitalized with Severe Leg Injur...          1
...    ...                                                ...        ...
1769  1412  Tom Brady and the Tampa Bay Buccaneers will de...          1
1770  1609  Schumer pledges quick delivery of $2,000 stimu...          0
1771  1079  The Gabby Petito Case: How Social Media Shaped...          1
1772  1811  "Not Guilty" Verdict Sparks Controversy: Lin W...          1
1773  1406  Coronavirus (COVID-19) Update: FDA Authorizes ...          0

[1774 rows x 3 columns]


### Display generated texts

In [66]:
pd.set_option('display.max_colwidth', 1000)
generated_only = concat_df.loc[concat_df['generated'] == 1]

print(generated_only['text'])
print(generated_only.text.str.len().sort_values(ascending = True))
print(generated_only.text.str.len().std())



2       Kamala Harris Tackles Root Causes of Migration from Central America, Amid Political Pressure\n\nCNN – Vice President Kamala Harris is focusing her efforts on addressing the root causes of migration from Central America, rather than managing the US southern border itself. This comes after President Joe Biden tasked her with leading efforts to stem migration from Mexico and the Northern Triangle countries.\n\nHarris and her team are cognizant of the political implications of her new role. They fear being linked to the border situation, as criticism from Republican critics and the media has portrayed her immigration role as a border assignment. However, the Vice President remains committed to finding long-term solutions.\n\nOn June 7, Harris will embark on a trip to Guatemala and Mexico to discuss issues driving Central American migration to the US. During her visit, she aims to secure commitments from private companies to invest in the Northern Triangle countries, which have some

### Display non-generated texts

In [55]:
non_generated = concat_df.loc[concat_df['generated'] == 0]
print(non_generated['text'])
print(non_generated.text.str.len().sort_values(ascending = True))
print(non_generated.text.str.len().std())

0       Kamala Harris to promote 'America is back' message in Singapore and Vietnam\n\nCNN — Vice President Kamala Harris is set to travel to Singapore and Vietnam later this month, becoming the highest-ranking Biden administration official to visit Asia so far.\n\n"Her trip will build on the Biden-Harris Administration's message to the world: America is back," her office said in a statement.\n\nThe statement continued, "Our Administration sees Asia as a critically important region in the world. The Vice President's visit will emphasize the importance of comprehensive engagement and strategic partnerships – key components of our Administration's approach to foreign policy."\n\nDuring meetings with government officials, the private sector, and civil society leaders, Harris will share the Biden administration's vision for a free and open Indo-Pacific, her office said, raise trade and security issues, including in the South China Sea, and promote economic cooperation.\n\nHarris will trave

In [126]:
targ_ser = targets_train['generated'].squeeze() #convert to series
pred = (text_train['text'].str.len()<1000).astype(int)
diff = targ_ser.compare(pred)
error_test = len(diff)/len(text_train)
f"Error: {error_test}"

'Error: 0.4391206313416009'

### Inspect validation set

In [127]:
targ_ser_v = targets_validation['generated'].squeeze() #convert to series
pred_v = (text_validation['text'].str.len()<1000).astype(int)
diff_v = targ_ser_v.compare(pred_v)
error = len(diff_v)/len(text_validation)
f"Error: {error}"

'Error: 0.44'

In [128]:
prediction = (
        (text_validation.set_index("id")["text"]
        .str.len()<1000).astype(int)
    )


# converting the prediction to the required format
prediction.name = "generated"
prediction = prediction.reset_index()

# saving the prediction
output_directory = get_output_directory(str(Path("authorship-verification-jupyter.ipynb").parent))
prediction.to_json(
        Path(output_directory) / "predictions.jsonl", orient="records", lines=True
)

